## Part 1: Data Cleaning

I have gotten response from 4 resources: Craigslist, Nextdoor, Facebook, and sublet.com. The communication format for each resource are:
* Craigslist: email and text message
    * Format: mbox for email, excel for text messages
* Nextdoor: Nextdoor inbox
    * Format: excel
* Facebook: Facebook messages
    * Format: JSON file for each conversation
* sublet.com: sublet.com inbox
    * Format: excel

### Section 1: emails from Craigslist

In [3]:
import pandas as pd
import re

In [4]:
#Import Craigslist data
#The emails were downloaded from gmail containing keyword "Craigslist" in mbox form
import mailbox

mbox = mailbox.mbox('\Data_Science_Coursework\Rental Data Analysis\Takeout\Mail\Craigslist.mbox')

In [5]:
#getting plain text 'email body'
def getbody(message): 
    body = None
    if message.is_multipart():
        for part in message.walk():
            if part.is_multipart():
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        body = subpart.get_payload(decode=True)
            elif part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True)
    elif message.get_content_type() == 'text/plain':
        body = message.get_payload(decode=True)
    return body

In [7]:
# Take a look at all the dates in the emails
for message in mbox:
   print(message['date'])

Thu, 26 Mar 2015 20:56:53 +0000 (UTC)
Thu, 26 Mar 2015 14:38:26 -0500
Sun, 4 Jun 2017 17:58:12 -0500 (CDT)
Sun, 20 May 2018 19:50:27 -0500 (CDT)
Sun, 20 Aug 2017 17:45:26 -0500 (CDT)
Sat, 09 Jun 2018 03:13:18 +0000
Wed, 1 Jun 2016 13:21:14 -0500
Sun, 6 Sep 2015 20:51:43 -0500 (CDT)
Tue, 24 Mar 2020 17:20:34 +0000 (GMT)
Mon, 16 Dec 2019 18:37:47 -0600
Mon, 16 Dec 2019 17:39:30 -0600
Mon, 16 Dec 2019 17:26:33 -0600
Sat, 9 May 2020 16:42:16 -0500
Wed, 10 Jul 2019 13:19:28 +0000 (UTC)
Mon, 20 Nov 2017 07:21:24 -0600 (CST)
Wed, 13 May 2015 19:44:24 -0500
Sun, 10 May 2015 22:26:51 -0500 (CDT)
Sat, 18 Apr 2020 18:08:37 -0500
Fri, 15 May 2020 21:30:00 -0500
Fri, 15 May 2020 21:17:36 -0500
Sun, 15 Jan 2017 22:22:35 -0600 (CST)
Sun, 26 Jan 2020 03:59:05 +0000 (GMT)
Sat, 11 Apr 2020 20:04:54 +0000 (GMT)
Sat, 23 May 2015 16:05:06 -0500
Sat,  4 Apr 2020 19:59:43 +0000 (GMT)
Sat,  4 Apr 2020 19:52:57 +0000 (GMT)
Sat,  9 May 2020 14:27:56 +0000 (GMT)
Wed, 30 Apr 2014 15:18:00 -0500
Sun,  8 Dec 2019 0

In [8]:
#Create a dataframe called email to store all email information.
email = pd.DataFrame(columns=('Name','Weekday','Date','Time','Source','Content','Visit'))

In [9]:
# Fill in the email dataframe from data from the mbox
i=0
for message in mbox:
    if '2020' in message['Date'] and 'Zixing' not in message['from'] and 'Ramzi' not in message['from'] \
    and 'Hunter' not in message['from']:
       email.loc[i,'Name'] = message['from']
       email.loc[i,'Date'] = message['date']
       email.loc[i,'Source'] = 'Craigslist'
       email.loc[i,'Content'] = getbody(message)
       email.loc[i,'Visit'] = False

       i=i+1
                          
email.head(20)

,Name,Weekday,Date,Time,Source,Content,Visit
0,"""craigslist - automated message, do not reply""...",NaN,"Tue, 24 Mar 2020 17:20:34 +0000 (GMT)",NaN,Craigslist,b'\nIMPORTANT - FURTHER ACTION IS REQUIRED TO ...,False
1,"""Aarice Dumas"" <3587dc348c8d32e887248eb50306bd...",NaN,"Sat, 9 May 2020 16:42:16 -0500",NaN,Craigslist,b'https://houston.craigslist.org/sub/d/houston...,False
2,"""Jessie Williamson"" <3a2b708c9d0035feb84d42d5b...",NaN,"Sat, 18 Apr 2020 18:08:37 -0500",NaN,Craigslist,"b'\nhi, i have a 6 pound chiweenie. i need him...",False
3,"""Zoe Nanson"" <22ea2bb690f93731aa598adb4203c447...",NaN,"Fri, 15 May 2020 21:17:36 -0500",NaN,Craigslist,"b'Hello! My name is Zoe, and my roommate and I...",False
4,"""craigslist - automated message, do not reply""...",NaN,"Sun, 26 Jan 2020 03:59:05 +0000 (GMT)",NaN,Craigslist,b'\nIMPORTANT - FURTHER ACTION IS REQUIRED TO ...,False
5,"""craigslist - automated message, do not reply""...",NaN,"Sat, 11 Apr 2020 20:04:54 +0000 (GMT)",NaN,Craigslist,b'This email contains:\n\n1) information about...,False
6,"""craigslist - automated message, do not reply""...",NaN,"Sat, 4 Apr 2020 19:59:43 +0000 (GMT)",NaN,Craigslist,b'PRIVATE! DO NOT FORWARD!\n\nAny request to s...,False
7,"""craigslist - automated message, do not reply""...",NaN,"Sat, 4 Apr 2020 19:52:57 +0000 (GMT)",NaN,Craigslist,b'PRIVATE! DO NOT FORWARD!\n\nAny request to s...,False
8,"""craigslist - automated message, do not reply""...",NaN,"Sat, 9 May 2020 14:27:56 +0000 (GMT)",NaN,Craigslist,"b'\nPosting ID # 7121230543:\n\n""First Month F...",False
9,"""Jon-Amerin Vorabutra"" <0a68beeefa203ea3b06acf...",NaN,"Sun, 17 May 2020 11:06:14 -0700",NaN,Craigslist,"b'Hello,\n\nI came across your listing on crai...",False


In [10]:
# See how many entries we have from craigslist
email.shape

(14, 7)

We have 14 emails from Craigslist regarding the apartment.

In [11]:
# Clean up the 'Name' column, leaving only the sender name in "".
for i, sender in enumerate(email['Name']):
    email.loc[i,'Name'] = re.findall(r'"([^"]*)"', sender)

There are two types of emails from Craigslist: 
1. emails from people inquiring about the apartment advertisement
2. emails from Craigslist telling me I have updated the listings

In [12]:
#if the sender is from craigslist, then it is an email about post update instead of an interested party.
for i in range(14):
    if 'craigslist' in str(email.loc[i, 'Name']):
        email.loc[i, 'Source'] = 'Craigslist_Update'

In [13]:
import datetime
from dateutil.parser import parse

In [14]:
#Split the column 'Date' containing weekday, date and time information into date, weekday, time, timezone columns.
for i in range(14):
    dt = parse(str(email.loc[i, 'Date']))
    email.loc[i, 'Weekday'] = str(email.loc[i, 'Date'])[0:3]
    email.loc[i, 'Date2'] = dt.date()
    email.loc[i, 'Time'] = dt.time()
    email.loc[i, 'TZ'] = dt. tzinfo

In [15]:
email.head(20)

,Name,Weekday,Date,Time,Source,Content,Visit,Date2,TZ
0,"[craigslist - automated message, do not reply]",Tue,"Tue, 24 Mar 2020 17:20:34 +0000 (GMT)",17:20:34,Craigslist_Update,b'\nIMPORTANT - FURTHER ACTION IS REQUIRED TO ...,False,2020-03-24,tzutc()
1,[Aarice Dumas],Sat,"Sat, 9 May 2020 16:42:16 -0500",16:42:16,Craigslist,b'https://houston.craigslist.org/sub/d/houston...,False,2020-05-09,"tzoffset(None, -18000)"
2,[Jessie Williamson],Sat,"Sat, 18 Apr 2020 18:08:37 -0500",18:08:37,Craigslist,"b'\nhi, i have a 6 pound chiweenie. i need him...",False,2020-04-18,"tzoffset(None, -18000)"
3,[Zoe Nanson],Fri,"Fri, 15 May 2020 21:17:36 -0500",21:17:36,Craigslist,"b'Hello! My name is Zoe, and my roommate and I...",False,2020-05-15,"tzoffset(None, -18000)"
4,"[craigslist - automated message, do not reply]",Sun,"Sun, 26 Jan 2020 03:59:05 +0000 (GMT)",03:59:05,Craigslist_Update,b'\nIMPORTANT - FURTHER ACTION IS REQUIRED TO ...,False,2020-01-26,tzutc()
5,"[craigslist - automated message, do not reply]",Sat,"Sat, 11 Apr 2020 20:04:54 +0000 (GMT)",20:04:54,Craigslist_Update,b'This email contains:\n\n1) information about...,False,2020-04-11,tzutc()
6,"[craigslist - automated message, do not reply]",Sat,"Sat, 4 Apr 2020 19:59:43 +0000 (GMT)",19:59:43,Craigslist_Update,b'PRIVATE! DO NOT FORWARD!\n\nAny request to s...,False,2020-04-04,tzutc()
7,"[craigslist - automated message, do not reply]",Sat,"Sat, 4 Apr 2020 19:52:57 +0000 (GMT)",19:52:57,Craigslist_Update,b'PRIVATE! DO NOT FORWARD!\n\nAny request to s...,False,2020-04-04,tzutc()
8,"[craigslist - automated message, do not reply]",Sat,"Sat, 9 May 2020 14:27:56 +0000 (GMT)",14:27:56,Craigslist_Update,"b'\nPosting ID # 7121230543:\n\n""First Month F...",False,2020-05-09,tzutc()
9,[Jon-Amerin Vorabutra],Sun,"Sun, 17 May 2020 11:06:14 -0700",11:06:14,Craigslist,"b'Hello,\n\nI came across your listing on crai...",False,2020-05-17,"tzoffset(None, -25200)"


In [16]:
# Drop the unnecessary columns
email.drop(columns=['Date','TZ'], axis=1, inplace=True)
email.rename(columns = {'Date2':'Date'}, inplace=True)
#Rearrange the columns
columns = ['Name', 'Weekday','Date', 'Time','Source','Content','Visit']
email = email[columns]
email.head()

,Name,Weekday,Date,Time,Source,Content,Visit
0,"[craigslist - automated message, do not reply]",Tue,2020-03-24,17:20:34,Craigslist_Update,b'\nIMPORTANT - FURTHER ACTION IS REQUIRED TO ...,False
1,[Aarice Dumas],Sat,2020-05-09,16:42:16,Craigslist,b'https://houston.craigslist.org/sub/d/houston...,False
2,[Jessie Williamson],Sat,2020-04-18,18:08:37,Craigslist,"b'\nhi, i have a 6 pound chiweenie. i need him...",False
3,[Zoe Nanson],Fri,2020-05-15,21:17:36,Craigslist,"b'Hello! My name is Zoe, and my roommate and I...",False
4,"[craigslist - automated message, do not reply]",Sun,2020-01-26,03:59:05,Craigslist_Update,b'\nIMPORTANT - FURTHER ACTION IS REQUIRED TO ...,False


In [17]:
 # One person contacted through email from Craigslist ended up visiting.
email.loc[10, 'Visit'] = True

In [18]:
# Save the created dataframe as email.csv
email.to_csv('E:\Data_Science_Coursework\Rental Data Analysis\email.csv')